In [1]:
from pandas import *
from collections import Counter

In [3]:
# algorithmes = ['K-Means', 'Expectation-Maximization']
# embeddings = ['One-Hot', 'Sentence transformers']
# features = ['100%'] # (100% = aucune réduction de dimensionnalité)
# clusters = [50, 100, 150, 200, 500, 1000, 5000]

# results = []
# for algorithme in algorithmes:
#     for embedding in embeddings:
#         for feature in features:
#             for cluster in clusters:
#                 results.append(\
#                 {'algorithme' : algorithme,\
#                     'embedding': embedding, \
#                     'N features': feature, \
#                     'K (nb clusters)' : cluster,\
#                     'Score Silhouette': None})


# # On va remplir ce dictionnaire avec les bons scores au fur et à mesure qu'on expérimente
# results = DataFrame(results)
# results

,algorithme,embedding,N features,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,100%,50,None
1,K-Means,One-Hot,100%,100,None
2,K-Means,One-Hot,100%,150,None
3,K-Means,One-Hot,100%,200,None
4,K-Means,One-Hot,100%,500,None
5,K-Means,One-Hot,100%,1000,None
6,K-Means,One-Hot,100%,5000,None
7,K-Means,Sentence transformers,100%,50,None
8,K-Means,Sentence transformers,100%,100,None
9,K-Means,Sentence transformers,100%,150,None


In [10]:
file_path = '../06-clustering/candidate_terms.csv'
with open(file_path, encoding='utf-8') as f:
    df = read_csv(f).drop(columns=["Unnamed: 0"])
    df['Terme'] = df['Terme'].astype('str')
    df['TF + DF'] = df['TF'] + df['DF']

df

,Corpora,Terme,Structure syntaxique,Forme lemmatisée,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm,Log Likelihood,TF,DF,TF*IDF,OKapiBM25,TF + DF
0,"['chum', 'chuqc', 'chusj', 'cisss_ca', 'cisss_...",services sociaux,NOM ADJ,service social,True,D012947,Services sociaux et travail social (activité),Social Work,True,1674.908057,40189,15418,1.000000,26.361483,55607
1,"['chum', 'chuqc', 'cisss_ca', 'cisss_cotenord'...",santé publique,NOM ADJ,santé public,True,D011634,Santé publique,Public Health,True,1572.987576,32510,11194,1.000000,18.947189,43704
2,"['chum', 'chuqc', 'chusj', 'cisss_ca', 'cisss_...",santé mentale,NOM ADJ,santé mental,True,D008603,Santé mentale,Mental Health,True,1579.080827,13229,4795,1.000000,24.142062,18024
3,"['chuqc', 'chusj', 'cisss_ca', 'cisss_cotenord...",ministère de la santé,NOM PRP DET:ART NOM,ministère de le santé,False,NaN,NaN,NaN,False,1411.378559,10741,7142,0.553007,21.734060,17883
4,"['chuqc', 'chusj', 'cisss_ca', 'cisss_cotenord...",ministère de la santé et des services,NOM PRP DET:ART NOM KON PRP:det NOM,ministère de le santé et des service,False,NaN,NaN,NaN,False,1107.888160,10560,7061,0.553007,31.111185,17621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10355,['santeestrie'],centre de crise,NOM PRP NOM,centre de crise,False,NaN,NaN,NaN,True,84.828851,3,3,0.136894,8.582629,6
10356,['laval_sante'],commotion cérébrale,NOM ADJ,commotion cérébral,True,D001924,Commotion de l'encéphale,Brain Concussion,True,388.209903,3,3,0.557684,13.725435,6
10357,['inesss'],carcinome rénal,NOM ADJ,carcinome rénal,True,D002292,Néphrocarcinome,"Carcinoma, Renal Cell",False,61.813956,3,3,0.141561,18.359316,6
10358,['inesss'],durée de vie,NOM PRP NOM,durée de vie,True,D008136,Longévité,Longevity,False,434.083437,3,3,0.184051,21.441105,6


In [11]:
from nltk.tokenize import RegexpTokenizer
regex = "[\w+-]+|\([\s+\w+\d+-]+\)|\w+|\w"
tokenizex = RegexpTokenizer(regex)

file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]

def to_tokens(kw, min_chars=2):
    tokens = tokenizex.tokenize(str(kw)) # split the string into a list of words
    tokens = [word for word in tokens if len(word) > min_chars] 
    tokens = [str(word) for word in tokens if word not in stopwords] 
    
    tokens = set(tokens) # to remove duplicates
    tokens = sorted(tokens) # converts our set back to a list and sorts words in alphabetical order
    return tokens

In [12]:
df['tokens'] = df["Terme"].apply(lambda x: to_tokens(
    x,
    min_chars=2,
)).astype(str)

df[['Terme', 'tokens']]

,Terme,tokens
0,services sociaux,"['services', 'sociaux']"
1,santé publique,"['publique', 'santé']"
2,santé mentale,"['mentale', 'santé']"
3,ministère de la santé,"['ministère', 'santé']"
4,ministère de la santé et des services,"['ministère', 'santé', 'services']"
...,...,...
10355,centre de crise,"['centre', 'crise']"
10356,commotion cérébrale,"['commotion', 'cérébrale']"
10357,carcinome rénal,"['carcinome', 'rénal']"
10358,durée de vie,"['durée', 'vie']"


In [13]:
vocab = sorted(set(df["tokens"].explode()))
len(vocab)

dim = len(vocab)
dim

9976

In [14]:
def to_vector(keyword,vocab):
    """
    Calculates vector of keyword on given vocabulary.

    Returns vector as a list of values.  
    """
    vector = []
    for word in vocab:
        if word in keyword:
            vector.append(1)
        else:
            vector.append(0)
    return vector

## **Kmeans**

### One-Hot embedding

In [17]:
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

algo = 'K-Means'
embed = 'One-Hot'

# for ratio in features:
#     r = float(ratio.strip('%'))/100
#     n = round(r * dim)
#     counter = Counter(df["tokens"].explode().to_list())
#     vocab = []
#     for key,value in counter.most_common(n):
#         vocab.append(key)
    
df["vector"] = df["tokens"].apply(lambda x: to_vector(x,vocab))

X = df['vector'].tolist()
kmeans = KMeans(n_clusters=k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
labels = kmeans.labels_

score  =  silhouette_score(X, labels)
results.loc[((results['algorithme'] == algo) & \
            (results['N features'] == ratio) & \
            (results['K (nb clusters)'] == k) & \
            (results['embedding'] == embed)), 'Score Silhouette'] = score

# results
print(k, score)

29 0.0012640075453869773


### Sentence transformers embedding

In [ ]:
# On va utiliser un modèle BERT/sentence transformers (fr) pour extraire nos embeddings plutôt que des simples one-hot encoding
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-base")

sentences = df['Terme'].tolist()
embeddings_st = model.encode(sentences, convert_to_numpy=True)

In [ ]:
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

for ratio in features:
    r = float(ratio.strip('%'))/100
    n = round(r * dim)
    counter = Counter(df["tokens"].explode().to_list())
    vocab = []
    for key,value in counter.most_common(n):
        vocab.append(key)
    
    df["vector"] = df["tokens"].apply(lambda x: to_vector(x,vocab))

    X = embeddings_st

    ###### APPLIQUER PCA AVEC UN NB DE DIMENSION CORRESPONDANT À R POUR FAIRE VARIER LA DIMENSION DES VECTEURS #########

    for k in clusters:
        kmeans = KMeans(n_clusters=k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)

        score  =  silhouette_score(X, kmeans.labels_, metric='cosine')
        results.loc[(results['algorithme'] == 'K-Means') & \
                    ((results['N features'] == ratio) & \
                    (results['K (nb clusters)'] == k) & \
                    results['embedding' == 'Sentence transformers']), 'Score Silhouette'] = score

results
    

## **Expectation-Maximization**

In [ ]:
from sklearn.mixture import GaussianMixture

### One-Hot embedding

In [ ]:
for ratio in features:
    r = float(ratio.strip('%'))/100
    n = round(r * dim)
    counter = Counter(df["tokens"].explode().to_list())
    vocab = []
    for key,value in counter.most_common(n):
        vocab.append(key)
    
    df["vector"] = df["tokens"].apply(lambda x: to_vector(x,vocab))

    X = df['vector'].tolist()
    for k in clusters:
        gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
        labels = gmm.predict(X)

        score  =  silhouette_score(X, labels)
        results.loc[((results['algorithme'] == 'Expectation-Maximization') &
                    (results['N features'] == ratio) & \
                    (results['K (nb clusters)'] == k) & \
                    (results['embedding'] == 'One-Hot')), 'Score Silhouette'] = score

results
    

### Sentence transformers embedding

In [ ]:
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

for ratio in features:
    r = float(ratio.strip('%'))/100
    n = round(r * dim)
    counter = Counter(df["tokens"].explode().to_list())
    vocab = []
    for key,value in counter.most_common(n):
        vocab.append(key)
    
    df["vector"] = df["tokens"].apply(lambda x: to_vector(x,vocab))

    X = embeddings_st

    ###### APPLIQUER PCA AVEC UN NB DE DIMENSION CORRESPONDANT À R POUR FAIRE VARIER LA DIMENSION DES VECTEURS #########

    for k in clusters:
        gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
        labels = gmm.predict(X)


        score  =  silhouette_score(X, labels, metric='cosine')
        results.loc[(results['algorithme'] == 'Expectation-Maximization') & \
                    ((results['N features'] == ratio) & \
                    (results['K (nb clusters)'] == k) & \
                    results['embedding' == 'Sentence transformers']), 'Score Silhouette'] = score

results